### Load in CSV Files

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)

In [2]:
pip install fuzzymatcher

Note: you may need to restart the kernel to use updated packages.


In [4]:
from fuzzymatcher import link_table, fuzzy_left_join

In [6]:
# descriptors of cannabis strains
can = pd.read_csv('/Users/stephaniekendall/Desktop/Flatiron/projects/Cannabis-Predictions/CSV Files/cannabis.csv', error_bad_lines=False)

# kushy csv
strains = pd.read_csv('/Users/stephaniekendall/Desktop/Flatiron/projects/Cannabis-Predictions/CSV Files/strains-kushy_api.2017-11-14.csv')

b'Skipping line 1333: expected 6 fields, saw 10\nSkipping line 1655: expected 6 fields, saw 10\nSkipping line 1656: expected 6 fields, saw 10\nSkipping line 1657: expected 6 fields, saw 10\nSkipping line 2102: expected 6 fields, saw 10\n'


In [7]:
can.head()

,strain,type,rating,effects,taste,description
0,100-og,hybrid,4,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",$100 OG is a 50/50 hybrid strain that packs a ...
1,98-white-widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...
2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody",1024 is a sativa-dominant hybrid bred in Spain...
3,13-dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",13 Dawgs is a hybrid of G13 and Chemdawg genet...
4,24k-gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%..."


In [8]:
can.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2348 entries, 0 to 2347
Data columns (total 6 columns):
strain         2347 non-null object
type           2347 non-null object
rating         2347 non-null object
effects        2347 non-null object
taste          2301 non-null object
description    2314 non-null object
dtypes: object(6)
memory usage: 110.1+ KB


In [9]:
# retrieve index of null values 
can.index[can.isnull().any(axis=1)]

Int64Index([ 103,  123,  160,  163,  215,  364,  368,  379,  484,  574,  594,
             673,  679,  736,  770,  817,  831,  856,  899,  900,  950,  956,
             965,  971, 1013, 1067, 1094, 1102, 1178, 1200, 1253, 1290, 1366,
            1367, 1369, 1441, 1477, 1512, 1513, 1514, 1543, 1545, 1546, 1554,
            1578, 1634, 1637, 1658, 1684, 1704, 1709, 1724, 1754, 1769, 1770,
            1799, 1851, 1858, 1914, 1963, 1970, 1991, 2089, 2098, 2113, 2127,
            2136, 2178, 2192, 2222, 2234, 2245, 2254, 2306, 2318],
           dtype='int64')

In [10]:
# remove - from strain names for matching with kushy data
can['strain'] = can['strain'].str.replace('-',' ')
can.head()

,strain,type,rating,effects,taste,description
0,100 og,hybrid,4,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",$100 OG is a 50/50 hybrid strain that packs a ...
1,98 white widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...
2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody",1024 is a sativa-dominant hybrid bred in Spain...
3,13 dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",13 Dawgs is a hybrid of G13 and Chemdawg genet...
4,24k gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%..."


In [11]:
can = can.dropna()
can = can.drop(columns=['description'])
can.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2273 entries, 0 to 2347
Data columns (total 5 columns):
strain     2273 non-null object
type       2273 non-null object
rating     2273 non-null object
effects    2273 non-null object
taste      2273 non-null object
dtypes: object(5)
memory usage: 106.5+ KB


In [12]:
strains.head()

,id,status,sort,name,slug,image,description,type,crosses,breeder,effects,ailment,flavor,location,terpenes,thc,thca,thcv,cbd,cbda,cbdv,cbn,cbg,cbgm,cbgv,cbc,cbcv,cbv,cbe,cbt,cbl
0,1,1,0,100 OG,NaN,NaN,<p>This strain is named after it's high price ...,Hybrid,NaN,Old School Breeder's Association,Focused,Depression,Citrus,NaN,Limonene,127,0.0,0.0,16,0.0,0.0,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,1,0,707 Headband,NaN,NaN,NaN,Hybrid,"378,272",Unknown Breeder,NaN,NaN,NaN,NaN,NaN,127,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1,0,A-10,NaN,NaN,NaN,Indica,0,NaN,"Relaxed, Happy, Uplifted, Energetic, Sleepy, D...","Stress, Insomnia, Pain, Muscle Spasms, Depression","Citrus, Sweet",NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1,0,Acapulco Gold,NaN,NaN,NaN,Sativa,0,NaN,"Happy, Euphoric, Uplifted, Relaxed, Creative, ...","Depression, Stress, Pain, Lack of Appetite","Earthy, Citrus",NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1,0,Afghani Bullrider,NaN,NaN,NaN,Hybrid,0,Unknown Breeder,"Uplifted, Relaxed, Happy, Euphoric, Dry Mouth,...","Stress, Depression, Insomnia, Pain","Sweet, Pine, Earthy",NaN,NaN,127,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
strains.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9524 entries, 0 to 9523
Data columns (total 31 columns):
id             9524 non-null int64
status         9524 non-null int64
sort           9524 non-null int64
name           9523 non-null object
slug           0 non-null float64
image          0 non-null float64
description    1 non-null object
type           9500 non-null object
crosses        564 non-null object
breeder        8979 non-null object
effects        1015 non-null object
ailment        971 non-null object
flavor         971 non-null object
location       621 non-null object
terpenes       1 non-null object
thc            9524 non-null int64
thca           8635 non-null float64
thcv           8635 non-null float64
cbd            9524 non-null int64
cbda           8635 non-null float64
cbdv           8635 non-null float64
cbn            9524 non-null int64
cbg            8635 non-null float64
cbgm           8635 non-null float64
cbgv           8635 non-null float64
cbc   

In [14]:
strains.describe()

,id,status,sort,slug,image,thc,thca,thcv,cbd,cbda,cbdv,cbn,cbg,cbgm,cbgv,cbc,cbcv,cbv,cbe,cbt,cbl
count,9524.000000,9524.000000,9524.0,0.0,0.0,9524.000000,8635.0,8635.0,9524.000000,8635.0,8635.0,9524.000000,8635.0,8635.0,8635.0,8635.0,8635.0,8635.0,8635.0,8635.0,8635.0
mean,4762.500000,1.906552,0.0,NaN,NaN,106.347648,0.0,0.0,3.749895,0.0,0.0,364.421252,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,2749.486316,0.291075,0.0,NaN,NaN,421.578887,0.0,0.0,62.131418,0.0,0.0,6209.909450,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,1.000000,1.000000,0.0,NaN,NaN,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,2381.750000,2.000000,0.0,NaN,NaN,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,4762.500000,2.000000,0.0,NaN,NaN,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,7143.250000,2.000000,0.0,NaN,NaN,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,9524.000000,2.000000,0.0,NaN,NaN,7700.000000,0.0,0.0,1900.000000,0.0,0.0,190000.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
# drop columns with only zeros or NaN
strains = strains.drop(columns=['slug','image','thca','thcv','cbda','cbdv'])

# drop columns with over 80% NaNs
strains = strains.drop(columns=['description','crosses','ailment','flavor','location','terpenes','breeder'])

# drop last 7 columns of empty cbd values
strains = strains.iloc[:, 0:8]

# replace - with whitespace in name 
strains['name'] = strains['name'].str.replace('-',' ')

strains.head(20)

,id,status,sort,name,type,effects,thc,cbd
0,1,1,0,100 OG,Hybrid,Focused,127,16
1,2,1,0,707 Headband,Hybrid,NaN,127,0
2,3,1,0,A 10,Indica,"Relaxed, Happy, Uplifted, Energetic, Sleepy, D...",0,0
3,4,1,0,Acapulco Gold,Sativa,"Happy, Euphoric, Uplifted, Relaxed, Creative, ...",0,0
4,5,1,0,Afghani Bullrider,Hybrid,"Uplifted, Relaxed, Happy, Euphoric, Dry Mouth,...",127,0
5,6,1,0,Afghan Big Bud,Indica,"Euphoric, Happy, Talkative, Relaxed, Sleepy, D...",0,0
6,7,1,0,Afghan Diesel,Hybrid,NaN,0,0
7,8,1,0,Afghan Kush,Hybrid,NaN,1300,0
8,9,1,0,Afghani,Indica,"Relaxed, Sleepy, Euphoric, Happy, Uplifted, Dr...",0,0
9,10,1,0,Afghanica,Hybrid,NaN,0,0


In [16]:
strains.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9524 entries, 0 to 9523
Data columns (total 8 columns):
id         9524 non-null int64
status     9524 non-null int64
sort       9524 non-null int64
name       9523 non-null object
type       9500 non-null object
effects    1015 non-null object
thc        9524 non-null int64
cbd        9524 non-null int64
dtypes: int64(5), object(3)
memory usage: 595.3+ KB


In [17]:
# create new dataframe for effects of strains
effects = strains[['id','name','type','effects']]
# strains = strains.drop(columns=['effects'])
effects.head(20)

,id,name,type,effects
0,1,100 OG,Hybrid,Focused
1,2,707 Headband,Hybrid,NaN
2,3,A 10,Indica,"Relaxed, Happy, Uplifted, Energetic, Sleepy, D..."
3,4,Acapulco Gold,Sativa,"Happy, Euphoric, Uplifted, Relaxed, Creative, ..."
4,5,Afghani Bullrider,Hybrid,"Uplifted, Relaxed, Happy, Euphoric, Dry Mouth,..."
5,6,Afghan Big Bud,Indica,"Euphoric, Happy, Talkative, Relaxed, Sleepy, D..."
6,7,Afghan Diesel,Hybrid,NaN
7,8,Afghan Kush,Hybrid,NaN
8,9,Afghani,Indica,"Relaxed, Sleepy, Euphoric, Happy, Uplifted, Dr..."
9,10,Afghanica,Hybrid,NaN


In [18]:
strains = strains.dropna()

In [19]:
effects.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9524 entries, 0 to 9523
Data columns (total 4 columns):
id         9524 non-null int64
name       9523 non-null object
type       9500 non-null object
effects    1015 non-null object
dtypes: int64(1), object(3)
memory usage: 297.7+ KB


In [20]:
effects = effects.dropna()
effects.tail()

,id,name,type,effects
2173,2174,X Wing,Indica,"Happy, Sleepy, Relaxed, Creative, Hungry, Dry ..."
2174,2175,XXX 420,Indica,"Relaxed, Talkative, Happy, Uplifted, Energetic..."
2175,2176,XXX OG,Indica,"Sleepy, Relaxed, Happy, Euphoric, Uplifted, Dr..."
2176,2177,Zkittlez,Indica,"Relaxed, Happy, Euphoric, Uplifted, Creative, ..."
2177,2178,Zoom Pie,Indica,"Hungry, Uplifted, Relaxed, Happy"


In [21]:
# make strain names & type lowercase for merging
effects['name'] = effects['name'].str.lower()
effects['type'] = effects['type'].str.lower()

In [22]:
effects.head()

,id,name,type,effects
0,1,100 og,hybrid,Focused
2,3,a 10,indica,"Relaxed, Happy, Uplifted, Energetic, Sleepy, D..."
3,4,acapulco gold,sativa,"Happy, Euphoric, Uplifted, Relaxed, Creative, ..."
4,5,afghani bullrider,hybrid,"Uplifted, Relaxed, Happy, Euphoric, Dry Mouth,..."
5,6,afghan big bud,indica,"Euphoric, Happy, Talkative, Relaxed, Sleepy, D..."


In [23]:
can.head()

,strain,type,rating,effects,taste
0,100 og,hybrid,4,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus"
1,98 white widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel"
2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody"
3,13 dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit"
4,24k gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange"


In [38]:
len(can)

2273

In [39]:
len(effects)

1015

In [28]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    """
    df_1 is the left table to join
    df_2 is the right table to join
    key1 is the key column of the left table
    key2 is the key column of the right table
    threshold is how close the matches should be to return a match, based on Levenshtein distance
    limit is the amount of matches that will get returned, these are sorted high to low
    """
    s = df_2[key2].tolist()

    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m

    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2

    return df_1

In [32]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
fuzzy_merge(effects, can, 'name', 'strain', 95)

,id,name,type,effects,matches
0,1,100 og,hybrid,Focused,100 og
2,3,a 10,indica,"Relaxed, Happy, Uplifted, Energetic, Sleepy, D...",a 10
3,4,acapulco gold,sativa,"Happy, Euphoric, Uplifted, Relaxed, Creative, ...",acapulco gold
4,5,afghani bullrider,hybrid,"Uplifted, Relaxed, Happy, Euphoric, Dry Mouth,...",
5,6,afghan big bud,indica,"Euphoric, Happy, Talkative, Relaxed, Sleepy, D...",afghan big bud
8,9,afghani,indica,"Relaxed, Sleepy, Euphoric, Happy, Uplifted, Dr...",afghani
10,11,afghooey,indica,"Uplifted, Sleepy, Euphoric, Relaxed, Talkative...",afghooey
11,12,afgoo,indica,"Relaxed, Sleepy, Happy, Euphoric, Hungry, Dry ...",afgoo
12,13,afgooey,indica,"Relaxed, Happy, Sleepy, Euphoric, Dry Mouth, P...",afgooey
17,18,alaskan thunder fuck,sativa,"Happy, Euphoric, Uplifted, Energetic, Relaxed,...",alaskan thunder fuck


In [35]:
# pip install fuzzywuzzy
from fuzzywuzzy import fuzz

def match_name(name, list_names, min_score=0):
    # -1 score incase we don't get any matches
    max_score = -1
    # Returning empty name for no match as well
    max_name = ""
    # Iternating over all names in the other
    for name2 in list_names:
        #Finding fuzzy match score
        score = fuzz.ratio(name, name2)
        # Checking if we are above our threshold and have a better score
        if (score > min_score) & (score > max_score):
            max_name = name2
            max_score = score
    return (max_name, max_score)

In [45]:
# List for dicts for easy dataframe creation
dict_list = []
# iterating over our players without salaries found above
for name in can.strain:
    # Use our method to find best match, we can set a threshold here
    match = match_name(name, effects.name, 95)
    
    # New dict for storing data
    dict_ = {}
    dict_.update({"strain" : name})
    dict_.update({"name" : match[0]})
    dict_list.append(dict_)
    
merge_table = pd.DataFrame(dict_list)
# Display results
merge_table

,name,strain
0,100 og,100 og
1,,98 white widow
2,1024,1024
3,,13 dawgs
4,,24k gold
5,,3 bears og
6,,3 kings
7,303 og,303 og
8,3d cbd,3d cbd
9,3x crazy,3x crazy


In [42]:
df = pd.merge(can, effects, left_on='strain', right_on='name')
df.head()

,strain,type_x,rating,effects_x,taste,id,name,type_y,effects_y,matches
0,100 og,hybrid,4,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",1,100 og,hybrid,Focused,100 og
1,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody",1415,1024,hybrid,"Uplifted, Relaxed, Energetic, Creative, Dry Mo...",1024
2,303 og,indica,4.2,"Relaxed,Happy,Euphoric,Uplifted,Giggly","Citrus,Pungent,Earthy",1705,303 og,indica,"Relaxed, Happy, Focused, Hungry, Euphoric, Dry...",303 og
3,3d cbd,sativa,4.6,"Uplifted,Focused,Happy,Talkative,Relaxed","Earthy,Woody,Flowery",1416,3d cbd,sativa,"Relaxed, Happy, Uplifted, Sleepy, Tingly, Dry ...",3d cbd
4,3x crazy,indica,4.4,"Relaxed,Tingly,Happy,Euphoric,Uplifted","Earthy,Grape,Sweet",1706,3x crazy,indica,"Relaxed, Happy, Uplifted, Euphoric, Creative, ...",3x crazy


In [44]:
len(df)

937

In [21]:
df1 = df.drop(columns=['type_y','description','strain'])
df1.head(20)

,id,name,type_x,effects_x,rating,effects_y,taste
0,1,100 og,hybrid,Focused,4,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus"
1,3,a 10,indica,"Relaxed, Happy, Uplifted, Energetic, Sleepy, D...",3.8,"Relaxed,Happy,Sleepy,Uplifted,Energetic","Citrus,Pungent,Sweet"
2,4,acapulco gold,sativa,"Happy, Euphoric, Uplifted, Relaxed, Creative, ...",4.5,"Happy,Uplifted,Euphoric,Energetic,Relaxed","Earthy,Sweet,Pungent"
3,6,afghan big bud,indica,"Euphoric, Happy, Talkative, Relaxed, Sleepy, D...",4,"Euphoric,Happy,Relaxed,Sleepy,Talkative","Pungent,Lemon,Peach"
4,9,afghani,indica,"Relaxed, Sleepy, Euphoric, Happy, Uplifted, Dr...",4.3,"Relaxed,Sleepy,Happy,Euphoric,Hungry","Earthy,Sweet,Pine"
5,11,afghooey,indica,"Uplifted, Sleepy, Euphoric, Relaxed, Talkative...",4.1,"Uplifted,Euphoric,Relaxed,Talkative,Creative","Earthy,Sweet,Woody"
6,12,afgoo,indica,"Relaxed, Sleepy, Happy, Euphoric, Hungry, Dry ...",4.1,"Relaxed,Sleepy,Happy,Euphoric,Hungry","Sweet,Pine,Woody"
7,13,afgooey,indica,"Relaxed, Happy, Sleepy, Euphoric, Dry Mouth, P...",4.3,"Relaxed,Happy,Sleepy,Giggly,Uplifted","Woody,Pine,Pungent"
8,18,alaskan thunder fuck,sativa,"Happy, Euphoric, Uplifted, Energetic, Relaxed,...",4.4,"Happy,Euphoric,Relaxed,Uplifted,Energetic","Earthy,Woody,Pine"
9,20,aloha,hybrid,"Energetic, Uplifted, Happy, Creative, Focused,...",4.2,"Energetic,Uplifted,Happy,Creative,Focused","Tropical,Sweet,Citrus"
